In [270]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Gain dataset

In [271]:
enron = read.csv("../input/enron-poi/enron.csv")

In [272]:
head(enron)
print("The number of employees:")
nrow(enron)
enron$name
print("The size of poi tables:")
dim(enron)
print("The name of each col:")
colnames(enron)
print("The number of non-poi vs poi:")
table(enron$poi)
print("The info of $poi:")
summary(enron$poi)

# 2. Explore the data

## (1) Explore Features

In [273]:
summary(enron)
sapply(enron, typeof)

In [274]:
library("ggplot2")
summary(enron$salary)
ggplot(enron, aes(x = salary, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "salary", 
       x = "salary", 
       y = "Density", 
       col = "poi") +
  theme_light()

summary(enron$bonus)
ggplot(enron, aes(x = bonus, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "bonus", 
       x = "bonus", 
       y = "Density", 
       col = "poi") +
  theme_light()

summary(enron$total_payments)
ggplot(enron, aes(x = total_payments, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_payments", 
       x = "total_payments", 
       y = "Density", 
       col = "poi") +
  theme_light()

summary(enron$exercised_stock_options)
ggplot(enron, aes(x = exercised_stock_options, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "exercised_stock_options", 
       x = "exercised_stock_options", 
       y = "Density", 
       col = "poi") +
  theme_light()


summary(enron$total_stock_value)
ggplot(enron, aes(x = total_stock_value, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "total_stock_value", 
       x = "total_stock_value", 
       y = "Density", 
       col = "poi") +
  theme_light()


summary(enron$shared_receipt_with_poi)
ggplot(enron, aes(x = shared_receipt_with_poi, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "shared_receipt_with_poi", 
       x = "shared_receipt_with_poi", 
       y = "Density", 
       col = "poi") +
  theme_light()

In [275]:
library("ggplot2")
summary(enron$to_messages)
ggplot(enron, aes(x = to_messages, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "to_messages", 
       x = "to_messages", 
       y = "Density", 
       col = "Class") +
  theme_light()

summary(enron$from_messages)
ggplot(enron, aes(x = from_messages, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "from_messages", 
       x = "from_messages", 
       y = "Density", 
       col = "Class") +
  theme_light()

summary(enron$from_this_person_to_poi)
ggplot(enron, aes(x = from_this_person_to_poi, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "from_this_person_to_poi", 
       x = "from_this_person_to_poi", 
       y = "Density", 
       col = "Class") +
  theme_light()

summary(enron$from_poi_to_this_person)
ggplot(enron, aes(x = from_poi_to_this_person, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "from_poi_to_this_person", 
       x = "from_poi_to_this_person", 
       y = "Density", 
       col = "Class") +
  theme_light()

In [276]:
ggplot(enron, aes(x=salary, y=bonus)) + geom_point()

ggplot(enron, aes(x=salary, fill=factor(poi))) + geom_histogram(bins=100) + labs(y="No. of transactions", title="Distribution of amount by poi", fill="poi") + facet_grid(poi~., scale="free_y") + theme(plot.title=element_text(hjust=0.5))

ggplot(enron, aes(x=poi, y=salary)) + geom_boxplot()

ggplot(enron, aes(x=poi, y=bonus)) + geom_boxplot()


## (2) non-poi VS poi

In [277]:
non_poi = subset(enron, enron$poi=="False")
poi = subset(enron, enron$poi=="True")
dim(non_poi)
print("summary of non_poi")
summary(non_poi)
head(non_poi)
dim(poi)
print("summary of poi")
summary(poi)
head(poi)

In [278]:
# compare money
non_poi_money = non_poi[c('salary','bonus','exercised_stock_options','total_stock_value')]
dim(non_poi_money)
summary(non_poi_money)
head(non_poi_money)
poi_money = poi[c('salary','bonus','exercised_stock_options','total_stock_value','total_payments')]
dim(poi_money)
summary(poi_money)
head(poi_money)

In [279]:
# compare email
non_poi_eamil = non_poi[c('shared_receipt_with_poi','to_messages','from_messages','from_this_person_to_poi','from_poi_to_this_person')]
dim(non_poi_eamil)
summary(non_poi_eamil)
head(non_poi_eamil)
poi_eamil = poi[c('shared_receipt_with_poi','to_messages','from_messages','from_this_person_to_poi','from_poi_to_this_person')]
dim(poi_eamil)
summary(poi_eamil)
head(poi_eamil)

# 3. Clean and transform the data

## (1) add new fearure

In [280]:
# added feature, fraction of e-mails to and from poi
enron$fraction_to_poi = enron$from_this_person_to_poi/enron$from_messages
enron$fraction_from_poi = enron$from_poi_to_this_person/enron$to_messages
# delete from_this_person_to_poi, from_messages, from_poi_to_this_person, to_messages
enron <- enron[,c(-3,-14,-16,-22)]

colnames(enron)
dim(enron)
head(enron)

summary(enron$fraction_to_poi)
ggplot(enron, aes(x = fraction_to_poi, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "fraction_to_poi", 
       x = "fraction_to_poi", 
       y = "Density", 
       col = "Class") +
  theme_light()

summary(enron$fraction_from_poi)
ggplot(enron, aes(x = fraction_from_poi, fill = poi)) +
  geom_density(alpha = 0.5) + 
  labs(title = "fraction_from_poi", 
       x = "fraction_from_poi", 
       y = "Density", 
       col = "Class") +
  theme_light()

## (2) Exception handling

In [281]:
enron_high_salary = subset(enron, enron$bonus>=5600000|enron$salary>1000000)
enron_high_salary = enron_high_salary[c('name','salary','bonus','poi','exercised_stock_options','restricted_stock','total_stock_value')]
head(enron_high_salary)
#enron_high_salary = enron[order(enron$salary>),]

In [282]:
# find outliers
data5 <-enron[,c("name","salary","bonus","poi","total_payments","exercised_stock_options","total_stock_value")]
str(data5)
outlier1=subset(data5,salary>1000000)
outlier1

outlier2=subset(data5,bonus>6000000)
outlier2

outlier3=subset(data5,total_payments>100000000)
outlier3

**Remove outliers that corrupt the data remove datapoints that create noise**

In [283]:
# Delete the "LOCKHART EUGENE E" data lines, all na 
which(enron$name=="LOCKHART EUGENE E")
enron =enron%>%filter(name!='LOCKHART EUGENE E')
#which(enron$name=="LOCKHART EUGENE E")

In [284]:
colnames(enron)
dim(enron)

In [285]:
Max_fraction_to_poi = subset(enron, enron$fraction_to_poi=="1")
head(Max_fraction_to_poi)

## (3) remove useless feature

In [286]:
# high NA rate features
is.na(enron)
dim(enron)
print("the % of NA: 0.4132625995")
sum(is.na(enron))
print("the number of NA in salary:")
sum(is.na(enron$salary))
print("the number of NA in deferral_payments:")
sum(is.na(enron$deferral_payments))
print("the number of NA in restricted_stock_deferred:")
sum(is.na(enron$restricted_stock_deferred))
print("the number of NA in loan_advances:")
sum(is.na(enron$loan_advances))
print("the number of NA in director_fees:")
sum(is.na(enron$director_fees))

In [287]:
# take out 'name','email_address', because of uesless
# take out 'loan_advances', 'restricted_stock_deferred','deferral_payments','director_fees', because of missing values take out all 'loan_advances' because of missing values
enron <- enron[,c(-1,-3,-9,-13,-18,-21)]
dim(enron)

In [288]:
enron[is.na(enron)] <- 0

In [289]:
enron_final = enron
dim(enron_final)
colnames(enron_final)

# 4. Sampling

In [290]:
# using both sampling
library(caret)
library(ROSE)

enron_final$poi = as.factor(enron_final$poi)
train_index = createDataPartition(enron_final$poi, times = 1, p=0.8, list=F)
train_data = enron_final[train_index,]
test_data = enron_final[-train_index,]
dim(train_data)
dim(test_data)
head(train_data)
head(test_data)

train_un = train_data

# sampling
set.seed(167)
train_un = ovun.sample(poi ~.,data=train_data, p=0.5,method="both")$data

#train_un <- ROSE(poi ~ .,data=train_data)$data
table(train_data$poi)
table(train_un$poi)

# 5. Data modeling

## (1) Logical Regression Tree

In [291]:
# install.packages("tree")
library(rpart)
library(partykit)
library(grid)
library(libcoin)
library(mvtnorm)

rpartTrue1 = rpart(poi~.,data=train_un,maxdepth=2)
print(rpartTrue1)
rpartTrue2=as.party(rpartTrue1)
plot(rpartTrue2)

#调用predict函数生成测试数据集的分类表
predictions = predict(rpartTrue1,test_data,type = "class")
#调用table建立测试数据集的分类表
table(test_data$poi,predictions)
#调用caret包提供的confusionMatrix生成混淆矩阵
library(caret)
confusionMatrix(table(predictions,test_data$poi))

In [292]:
rpartTrue3 = rpart(poi~.,data=train_un,maxdepth=3)
print(rpartTrue3)
rpartTrue4=as.party(rpartTrue3)
plot(rpartTrue4)

#调用predict函数生成测试数据集的分类表
predictions = predict(rpartTrue3,test_data,type = "class")
#调用table建立测试数据集的分类表
table(test_data$poi,predictions)
#调用caret包提供的confusionMatrix生成混淆矩阵
library(caret)
confusionMatrix(table(predictions,test_data$poi))

In [293]:
rpartTrue5 = rpart(poi~.,data=train_un,maxdepth=1)
print(rpartTrue5)
rpartTrue6=as.party(rpartTrue5)
plot(rpartTrue6)
#调用predict函数生成测试数据集的分类表
predictions = predict(rpartTrue5,test_data,type = "class")
#调用table建立测试数据集的分类表
table(test_data$poi,predictions)
#调用caret包提供的confusionMatrix生成混淆矩阵
library(caret)
confusionMatrix(table(predictions,test_data$poi))

In [294]:
rpartTrue7 = rpart(poi~.,data=train_un,maxdepth=5)
print(rpartTrue7)
rpartTrue8=as.party(rpartTrue7)
plot(rpartTrue8)
#调用predict函数生成测试数据集的分类表
predictions = predict(rpartTrue7,test_data,type = "class")
#调用table建立测试数据集的分类表
table(test_data$poi,predictions)
#调用caret包提供的confusionMatrix生成混淆矩阵
library(caret)
confusionMatrix(table(predictions,test_data$poi))

## (2) Decision Tree

In [295]:
# install.packages('rpart')
# install.packages('rpart.plot')
library(rpart)
library(rpart.plot)

In [296]:
# index <- sample(nrow(enron_final), 0.8*nrow(enron_final))
# train <- enron_final[index, ]
# test <- enron_final[-index, ]

In [297]:
tc <- rpart.control(minbucket=5,maxdepth=10,xval=5,cp=0.005)
fit <- rpart(poi ~ ., data=train_un, control="tc")

In [298]:
train.pred <- predict(fit, train_un, type="class")
table(train_un$poi == train.pred)['TRUE'] / length(train.pred)
#调用table建立测试数据集的分类表
table(train_un$poi,train.pred)
#调用caret包提供的confusionMatrix生成混淆矩阵
confusionMatrix(table(train.pred,train_un$poi))

test.pred <- predict(fit, test_data, type="class") 
table(test_data$poi == test.pred)['TRUE'] / length(test.pred)
#调用table建立测试数据集的分类表
table(test_data$poi,test.pred)
#调用caret包提供的confusionMatrix生成混淆矩阵
confusionMatrix(table(test.pred,test_data$poi))


In [299]:
rpart.plot(fit, main="Decision Tree")

In [300]:
fit$cptable

## (3) Random Forest

In [301]:
# find mtry
library("randomForest")
n<-length(names(train_data))     #计算数据集中自变量个数，等同n=ncol(train_data)
rate=1     #设置模型误判率向量初始值

for(i in 1:(n-1)){
  set.seed(1234)
  rf_train<-randomForest(as.factor(train_un$poi)~.,data=train_un,mtry=i,ntree=1000)
  rate[i]<-mean(rf_train$err.rate)   #计算基于OOB数据的模型误判率均值
  print(rf_train)    
}
print("error rate for each model:")
rate     #展示所有模型误判率的均值
plot(rate)

In [302]:
# find ntree
#寻找最佳参数ntree，即指定随机森林所包含的最佳决策树数目
set.seed(100)
rf_train<-randomForest(as.factor(train_un$poi)~.,data=train_un,mtry=12,ntree=1000)
plot(rf_train)    #绘制模型误差与决策树数量关系图  
legend(800,0.02,"poi=0",cex=0.9,bty="n")    
legend(800,0.0245,"total",cex=0.09,bty="n")  

In [303]:
#install.packages('randomForest')
library(ROCR)
library(randomForest)
# train
model_rf = randomForest(poi ~.,data=train_un,proximity=TRUE, importance=TRUE)
model_rf
MDSplot(model_rf,train_un$poi)
print(model_rf)    #展示随机森林模型简要信息
hist(treesize(model_rf))   #展示随机森林模型中每棵决策树的节点数

In [304]:
model_rf$importance
varImpPlot(model_rf, main = "variable importance")
barplot(model_rf$importance[,1],main="Histogram of importance measures of input variables")
box()

In [320]:
library(Metrics)
library(pROC)
library(precrec)

rf_pred = predict(model_rf, newdata=test_data)
#confusionMatrix(test_data$poi,rf_pred, positive="1")
pls.cmx <- confusionMatrix(data = rf_pred, test_data$poi)
pls.cmx

plot(margin(rf_train,test_data$poi),main="the graph of the probability that observations are judged to be correct")

prediction_test = evalmod(scores = as.numeric(rf_pred), labels = test_data$poi, mode = "rocprc")
prediction_test

#绘制ROC曲线
ran_roc <- roc(test_data$poi,as.numeric(rf_pred))
plot(ran_roc, print.auc=TRUE, auc.polygon=TRUE, grid=c(0.1, 0.2),grid.col=c("green", "red"), max.auc.polygon=TRUE,auc.polygon.col="skyblue", print.thres=TRUE,main='ROC curve of random forest model')

## (4) Neural Network

In [311]:
library(nnet) 
err11=0
err12=0
n_tr=dim(train_un)[1]
n_te=dim(test_data)[1]
for(i in seq(1, 601, 100))
{
  model=nnet(poi ~ ., data=train_un,maxit=i,size=6,decay = 0.1)
  err11[i]=sum(predict(model,train_un,type='class')!=train_un$poi)/n_tr
  err12[i]=sum(predict(model,test_data,type='class')!=test_data$poi)/n_te
}

In [312]:
error_1 = na.omit(err11)
error_2 = na.omit(err12)
plot(seq(1, 601, 100),error_1,col=1,type="b",ylab="Error rate",xlab="Training epoch",ylim=c(min(min(error_1),min(error_2)),max(max(error_1),max(error_2))))
lines(seq(1, 601, 100),error_2,col=2,type="b")
legend("topleft",pch=c(15,15),legend=c("Train","Test"),col=c(1,2),bty="n")

In [315]:
#Final model and evaluation result
model_best=nnet(poi ~ ., data=train_un,maxit=300,size=6,decay = 0.1)

prediction_test = predict(model_best,test_data,type="class")

table1 = table(test_data$poi,prediction_test)
confusionMatrix(table1,positive="True")

In [318]:
library(pROC)
library(precrec)
#绘制ROC曲线
# ran_roc <- roc(test_data$fraud_reported,as.numeric(prediction_test))
# plot(ran_roc, print.auc=TRUE, auc.polygon=TRUE, grid=c(0.1, 0.2),grid.col=c("green", "red"), max.auc.polygon=TRUE,auc.polygon.col="skyblue", print.thres=TRUE,main='ROC curve of random forest model, Mtry =1,ntree=500')

prediction_test = evalmod(scores = as.numeric(prediction_test), labels = test_data$poi, mode = "rocprc")
prediction_test

## (5) K-flod Cross Validation

**a) Random Forest**

In [306]:
library(plyr)
CVgroup <- function(k,datasize,seed){
  cvlist <- list()
  set.seed(seed)
  n <- rep(1:k,ceiling(datasize/k))[1:datasize]    #将数据分成K份，并生成的完成数据集n
  temp <- sample(n,datasize)   #把n打乱
  x <- 1:k
  dataseq <- 1:datasize
  cvlist <- lapply(x,function(x) dataseq[temp==x])  #dataseq中随机生成k个随机有序数据列
  return(cvlist)
}
k <- 10
datasize <- nrow(iris)
cvlist <- CVgroup(k = k,datasize = datasize,seed = 1206)
cvlist
###
data <- enron
data = data %>% mutate_if(is.character,as.factor) 
data = as.data.frame(data)
pred <- data.frame()   #存储预测结果
                   
library(plyr)
library(randomForest)
m <- seq(60,500,by = 20)  #如果数据量大尽量间隔大点，间隔过小没有实际意义
for(j in m){   #j指的是随机森林的数量
  progress.bar <- create_progress_bar("text")  #plyr包中的create_progress_bar函数创建一个进度条，
  progress.bar$init(k)   #设置上面的任务数，几折就是几个任务
  for (i in 1:k){
    train <- data[-cvlist[[i]],]  #刚才通过cvgroup生成的函数
    test <- data[cvlist[[i]],]
    model <-randomForest(poi~.,data = train,ntree = j)   #建模，ntree=j 指的树数
    prediction <- predict(model,subset(test,select = -poi))   #预测
    randomtree <- rep(j,length(prediction))   #随机森林树的数量
    kcross <- rep(i,length(prediction))   #i是第几次循环交叉，共K次
    temp <- data.frame(cbind(subset(test,select = poi),prediction,randomtree,kcross))#真实值、预测值、随机森林树数、预测组编号捆绑在一起组成新的数据框tenp
    pred <- rbind(pred,temp)   #temp按行和pred合并
    print(paste("random forest:",j))  #循环至树数j的随机森林模型
    progress.bar$step() #输出进度条。告知完成了这个任务的百分之几
  }
}

In [307]:
#pred
print("the poi:")
table(pred$fraud_reported)
print("the prediction:")
table(pred$prediction)
summary(pred)
pred$result = (pred$poi == pred$poi)
print("the result of k-10 random forest:")
table(pred$result)
table(pred$result)/nrow(pred)